In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd

D:\Applications\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 数据预处理

## panda数据读取

In [2]:
train_data=pd.read_csv("D:\\jupterWork\\encoder\\ag_news_csv\\train.csv",header=None,usecols=[1,2])
train_labels=pd.read_csv("D:\\jupterWork\\encoder\\ag_news_csv\\train.csv",header=None,usecols=[0])
test_data=pd.read_csv("D:\\jupterWork\\encoder\\ag_news_csv\\test.csv",header=None,usecols=[1,2])
test_labels=pd.read_csv("D:\\jupterWork\\encoder\\ag_news_csv\\test.csv",header=None,usecols=[0])

## titlte+description

In [3]:
#array
train_data=train_data.values
y_train=train_labels.values
test_data=test_data.values
y_test=test_labels.values

train_messages = [None]*120000
test_messages = [None]*7600

for index, data in enumerate(train_data):
    train_messages[index] = data[0]+'.'+data[1]

for index, data in enumerate(test_data):
    test_messages[index] = data[0]+'.'+data[1]

## tokenizer词典 

In [4]:
tokenizer=keras.preprocessing.text.Tokenizer(num_words=10000, 
                                   filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', 
                                   lower=True, split=' ', 
                                   char_level=False, 
                                   oov_token=None)
tokenizer.fit_on_texts(train_messages)#词典

In [5]:
#print( 'word_counts:',tokenizer.word_counts) #单词出现次数
#print( '\nword_index:',tokenizer.word_index) #单词出现的位置
#print( '\nword_docs:',tokenizer.word_docs) #单词出现在的文档（行）数
#print( '\nindex_docs',tokenizer.index_docs) #word_index：word_docs

In [6]:
train_sequences = tokenizer.texts_to_sequences(train_messages)
test_sequences = tokenizer.texts_to_sequences(test_messages)
maxlen=0
max_index=0
for i, seq in enumerate(train_sequences):
    if len(seq)>maxlen:
        maxlen=len(seq)
    if max(seq)>max_index:
        max_index=max(seq)
print(maxlen)
print(max_index)

174
9999


In [7]:
train_x = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen)
test_x = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen)
y_train_labels = keras.utils.to_categorical(y_train, num_classes=5)
y_test_labels = keras.utils.to_categorical(y_test, num_classes=5)
train_x
#print( tokenizer.texts_to_matrix(test_messages))  # 矩阵化=one_hot

array([[   0,    0,    0, ..., 4049,  797,  332],
       [   0,    0,    0, ...,    4,    1,  128],
       [   0,    0,    0, ...,    4,    1, 1214],
       ...,
       [   0,    0,    0, ...,  346,   65,  123],
       [   0,    0,    0, ...,   42,   16, 1666],
       [   0,    0,    0, ..., 2095, 3435,   72]])

# basic cnn model

In [8]:
def base_cnn_model(y,max_features,embedding_dims,filters):
    kernel_size = 3
    model = keras.Sequential()
    model.add(keras.layers.Embedding(max_features, embedding_dims)) # 使用Embedding层将每个词编码转换为词向量(10000,181,128)
    model.add(keras.layers.Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))
    # 池化
    model.add(keras.layers.GlobalMaxPooling1D())
    model.add(keras.layers.Dense(y.shape[1], activation='softmax')) #第一个参数units: 全连接层输出的维度，即下一层神经元的个数。
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.summary()

    return model

In [9]:
max_features=10000
#max_features=70344
embedding_dims=128
filters=32
#y_train=np_utils.to_categorical(4)
model=base_cnn_model(y_train_labels,max_features,embedding_dims,filters)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 32)          12320     
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 1,292,485
Trainable params: 1,292,485
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(train_x,
                    y_train_labels,
                    validation_split=0.2,
                    batch_size=64,
                    epochs=3,
                    verbose=1,
                    shuffle=True)

D:\Applications\anaconda\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 96000 samples, validate on 24000 samples
Epoch 1/3
96000/96000 [==============================] - 234s 2ms/step - loss: 0.3950 - acc: 0.8736 - val_loss: 0.2859 - val_acc: 0.9018
Epoch 2/3
96000/96000 [==============================] - 223s 2ms/step - loss: 0.1953 - acc: 0.9341 - val_loss: 0.2863 - val_acc: 0.9019
Epoch 3/3
96000/96000 [==============================] - 231s 2ms/step - loss: 0.1273 - acc: 0.9578 - val_loss: 0.3065 - val_acc: 0.8975


In [11]:
results = model.evaluate(test_x, y_test_labels)
print(results)

7600/7600 [==============================] - 5s 595us/step
[0.2942187399299521, 0.9025]
